In [2]:
import findspark
findspark.init()
import pyspark

In [3]:
#NOTE: Links is 10x larger than Pages --> Partition using this
#NOTE: This is a DIRECTED GRAPH
#source_rdd will contain (key=character, value=a comic that the character is in)


#Adapted from https://github.com/thouis/SparkPageRank/blob/master/PageRank.py

if __name__ == '__main__':
    sc = pyspark.SparkContext("Spark")
    sc.setLogLevel('WARN')

    links = sc.textFile('s3://Harvard-CS205/wikipedia/links-simple-sorted.txt', 32)
    page_names = sc.textFile('s3://Harvard-CS205/wikipedia/titles-sorted.txt', 32)

    # process links into (node #, [neighbor node #, neighbor node #, ...]
    neighbor_graph = links.map(link_string_to_KV)

    # create an RDD for looking up page names from numbers
    # remember that it's all 1-indexed
    page_names = page_names.zipWithIndex().map(lambda (n, id): (id + 1, n))
    page_names = page_names.sortByKey().cache()

    #######################################################################
    # set up partitioning - we have roughly 16 workers, if we're on AWS with 4
    # nodes not counting the driver.  This is 16 partitions per worker.
    #
    # Cache this result, so we don't recompute the link_string_to_KV() each time.
    #######################################################################
    neighbor_graph = neighbor_graph.partitionBy(256).cache()

    # find Kevin Bacon
    Kevin_Bacon = page_names.filter(lambda (K, V): V == 'Kevin_Bacon').collect()
    # This should be [(node_id, 'Kevin_Bacon')]
    assert len(Kevin_Bacon) == 1
    Kevin_Bacon = Kevin_Bacon[0][0]  # extract node id

    # find Harvard University
    Harvard_University = page_names.filter(lambda (K, V):
                                           V == 'Harvard_University').collect()
    # This should be [(node_id, 'Harvard_University')]
    assert len(Harvard_University) == 1
    Harvard_University = Harvard_University[0][0]  # extract node id

    pagerank(neighbor_graph,
             page_names,
             iterations=10,
             inspect=[Kevin_Bacon, Harvard_University])

Py4JJavaError: An error occurred while calling o18.partitions.
: java.io.IOException: No FileSystem for scheme: s3
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:2584)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:2591)
	at org.apache.hadoop.fs.FileSystem.access$200(FileSystem.java:91)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:2630)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:2612)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:370)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:296)
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:256)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:228)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:313)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:207)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:239)
	